### <center> Resolution of Max-Cut using QAOA. </center>

<center> Author : Leder Aguirre </center>
<center> Date : 2023 </center>

### Table of contents

1. [Introduction](#introduction)
    1. [Problem](#problem)
    2. [algorithm](#algorithm)
2. [Importations](#importations)
3. [Toy example](#toy_example)
4. [Performance tests on random instances](#random_instances)
5. [Bibliography](#bibliography)
$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}
\renewcommand{\bra}[1]{\left\langle{#1}\right|}$

#### Problem

For an graph $G=(V,E)$ with edge weights $c_e$, the Max(imum) Cut problem is to find a cut $S \subset V$ such that the weight of edges accross $(S,\bar{S})$ is maximized. It can be formulated as the Quadratic Unconstrained Optimization Problem (QUBO):

Let $y_i$ be the decision variable, equal to 1 if the node $i$ belongs to $S$, -1 otherwise. The optimization problem is:

\begin{align}
\text{max} & \quad & \sum_{(i,j) \in E} w_{i,j} \left(\frac{1 - y_iy_j}{2} \right)\\
\text{s.t.} & \quad & y_i \in \{-1,1\} \quad \forall i \in V
\end{align}


#### Algorithm

QAOA is a variational algorithm that uses a unitary $\hat{U}(\beta, \gamma)$ characterized by the parameters $(\beta, \gamma) \in \mathbb{R}$ to prepare a quantum state $\ket{\psi\left(\beta, \gamma\right)}$ encoding the *optimal solution* to the combinatorial optimization problem (COP), according to one of the two equations below:

\begin{align}
		\ket{\psi\left(\beta, \gamma\right)} &= \ket{\psi_T\left(\beta, \gamma\right)} = \underbrace{\hat{U}(\beta)\hat{U}(\gamma) \dots \hat{U}(\beta)\hat{U}(\gamma)}_{\text{$T$ times}} \ket{\psi(t_0)}\\
        \ket{\psi\left(\beta, \gamma\right)} &= \hat{U}(\beta_T)\hat{U}(\gamma_T) \dots \hat{U}(\beta_1)\hat{U}(\gamma_1) \ket{\psi(t_0)}
\end{align}


- *Reason of why the preparded state encodes the optimal solution*: We start preparting the quantum state $\ket{\psi(t_0)}$ associated to the maximum eigenvalue of some "mixing" Hamiltonian $\hat{H}_B$. Then, through an adiabatic evolution associated to the parameter $T$, this state will evolve to the quantum state associated to the maximum eigenvalue of the "problem" Hamiltonian $\hat{H}_P$. The problem Hamiltonian is a diagonal operator whose eigenvalues are the values of all the possible solutions.
- *Differences between the two architectures (equations)*: The first equation is the original formulation in the sense that it is associated directly to an adiabatic evolution. In the second equation the dimension of the parameters is increased so that the depth of the circuits used might be reduced.

The algorithm to solve the COP is described by the following process:
	
1. Initialize $\beta, \gamma$ to suitable real values.
2. Repeat until some suitable convergence criteria is met:
    1. Prepare the state $\ket{\psi\left(\beta, \gamma\right)}$ using QAOA circuit.
    2. Compute the expectation $ \bra{\psi\left(\beta, \gamma\right)} \hat{H}_P \ket{\psi\left(\beta, \gamma\right)}$ via a sampling procedure.
    3. Find a better set of parameters $\left(\beta_{new}, \gamma_{new}\right)$ using a classical optimization algorithm.
    4. Set the current parameters $\left(\beta, \gamma\right)$ equal to the new parameters $\left(\beta_{new}, \gamma_{new}\right)$.